In [1]:
# Given the values of a single peakon on a coarse grid, this DNN predicts the value of the peakon at a particular location.
# Essentially, given the values shown by the @ and G symbols, and a location within the ... region, it will output its
# prediction for the value at that location.
# (In terms of the gen_one_data function below, then (t, x) is the location of the G symbol and (t2, x2) is the random
# location within the ... region.)
#
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @
#
#       .........
#       .       .
# @   @ . @   @ . @   @
#       .       .
#       .       .
#       .       .
# @   @ . G   @ . @   @
#       .       .
#       .........
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @

In [2]:
%run base/base_eq.ipynb
%run base/base_plot.ipynb
%run base/base_ml.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
class BilinearInterp(PointPredictorMixin, BilinearInterpBase):
    pass

In [ ]:
def gen_one_peakon_at_point():
    """Returns a (feature, label) pair, where the features are the values of
    a single-peakon solution on a coarse grid, and a particular point, and the 
    label is the value of the single-peakon solution at that particular point.
    """
    point, peakon = gen_one_peakon()
    return sol_at_point(point, peakon)

In [5]:
### DNN hyperparameters
hidden_units = [1000] * 20
# i.e. the number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
drop_rate = 0.4
drop_type = 'dropout'
preprocessor = ScaleDataOverall(momentum=0.999)
activation=tf.nn.elu
uuid = None

### Training hyperparameters #1
gradient_clip = 5.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 1000  # How many steps to print the current loss.
model_dir_base = './saved_models/point_single_peakon/'
model_dir = model_dir_str(model_dir_base, hidden_units, logits, 
                          drop_rate=drop_rate, drop_type=drop_type,
                          preprocessor=preprocessor, activation=activation, 
                          uuid=uuid)

dnn = create_dnn(hidden_units, logits, drop_rate=drop_rate,
                 drop_type=drop_type, model_dir=model_dir, 
                 log_steps=log_steps, activation=activation, 
                 gradient_clip=gradient_clip, batch_norm=False)
preprocessor.load(model_dir)

In [8]:
if TRAIN:
    ### Training hyperparameters #2
    batch_size = 30
    steps = 3000
    train_input_fn = preprocessor.data(BatchData(gen_one_data=gen_one_peakon_at_point, batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])
    
    batch_size = 300
    steps = 6000
    train_input_fn = preprocessor.data(BatchData(gen_one_data=gen_one_peakon_at_point, batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])
    
    batch_size = 1000
    steps = 12000
    train_input_fn = preprocessor.data(BatchData(gen_one_data=gen_one_peakon_at_point, batch_size=batch_size))
    dnn.train(input_fn=train_input_fn(use_tf=True), max_steps=steps, 
              hooks=[ProcessorSavingHook(preprocessor, model_dir)])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt-14619
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 14620 into ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt.
INFO:tensorflow:loss = 0.11425025, step = 14620
INFO:tensorflow:Saving checkpoints for 15395 into ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt.
INFO:tensorflow:global_step/sec: 1.28837
INFO:tensorflow:loss = 0.11744919, step = 15620 (776.179 sec)
INFO:tensorflow:Saving checkpoints for 16160 into ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt.
INFO:tensorflow:global_step/sec: 1.2754